In [52]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array , load_img
from tensorflow.keras.layers import Conv2D , Flatten , MaxPool2D , Dense , GlobalAveragePooling2D , AveragePooling2D , Dropout , Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

In [53]:
lr = 1e-4
epochs = 10
batch_size = 32

In [54]:
directory = '/content/drive/My Drive/csv files/dataset'
categories = ['with_mask' , 'without_mask']

In [55]:
data = []
labels = []

for category in categories:
  path = os.path.join(directory , category)
  for img in os.listdir(path):
    img_path = os.path.join(path , img)
    image = load_img(img_path , target_size=(224,224))
    image = img_to_array(image)

    data.append(image)
    labels.append(category)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [56]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [57]:
# convert data and labels from list to numpy arrays
data = np.array(data , dtype ='float32')
labels = np.array(labels)

In [58]:
#splitting the model into training and testing dataset
X_train , X_test, y_train , y_test = train_test_split(data , labels , test_size = 0.2 , random_state = 101)

In [59]:
datagen = ImageDataGenerator(rotation_range=20,
                              zoom_range=0.15,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.15,
                              horizontal_flip=True,
                              fill_mode="nearest",
                            )

In [60]:
base_model = MobileNetV2(weights='imagenet' , include_top= False , input_tensor=Input(shape = (224 , 224, 3)))

head_model = base_model.output
head_model= AveragePooling2D(7)(head_model)
head_model = Flatten()(head_model)
head_model = Dense( 128 , activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense( 2 , activation='softmax')(head_model)

In [61]:
model = Model(inputs = base_model.input , outputs = head_model)

for layer in base_model.layers:
  layer.trainable = False

In [62]:
optim = optimizers.Adam(learning_rate=lr , decay = lr/batch_size)
model.compile(optim , loss=tensorflow.keras.losses.binary_crossentropy , metrics=['accuracy'])

In [63]:
model.fit(datagen.flow(X_train , y_train, batch_size = batch_size) , 
                  steps_per_epoch=len(X_train)//batch_size , 
                  validation_data=(X_test , y_test) , 
                  validation_steps= len(X_test)//batch_size , 
                  epochs = epochs)

Epoch 1/10
95/95 [==============================] - 40s 425ms/step - loss: 0.6921 - accuracy: 0.6694 - val_loss: 0.3878 - val_accuracy: 0.8449
Epoch 2/10
95/95 [==============================] - 39s 412ms/step - loss: 0.4502 - accuracy: 0.7917 - val_loss: 0.3302 - val_accuracy: 0.8683
Epoch 3/10
95/95 [==============================] - 39s 410ms/step - loss: 0.3828 - accuracy: 0.8365 - val_loss: 0.3010 - val_accuracy: 0.8801
Epoch 4/10
95/95 [==============================] - 39s 408ms/step - loss: 0.3633 - accuracy: 0.8438 - val_loss: 0.2745 - val_accuracy: 0.8957
Epoch 5/10
95/95 [==============================] - 38s 405ms/step - loss: 0.3265 - accuracy: 0.8652 - val_loss: 0.2649 - val_accuracy: 0.8996
Epoch 6/10
95/95 [==============================] - 38s 403ms/step - loss: 0.3124 - accuracy: 0.8718 - val_loss: 0.2536 - val_accuracy: 0.9035
Epoch 7/10
95/95 [==============================] - 39s 413ms/step - loss: 0.3097 - accuracy: 0.8741 - val_loss: 0.2454 - val_accuracy: 0.9035

In [65]:
# serialize model to JSON
model_json = model.to_json()
with open("/content/drive/My Drive/Colab Notebooks/Tensorflow 2.x/mask detection/mask_detection.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Colab Notebooks/Tensorflow 2.x/mask detection/mask_detection.h5")
print("Saved model to disk")

Saved model to disk
